In [1]:
!pip install turicreate
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install sklearn

In [2]:
import turicreate as tc

In [3]:
sf = tc.SFrame.read_csv('ratings.csv', verbose = False)
sf = sf.remove_column('rating_id') #remove the db id
sf = sf.rename({'trail_id': 'item_id'}) #rinomiano il trail_id in item_id in maniera tale da conformarci con la libreria Turicreate
sf 

user_rating,item_id,user_id
5.0,273,274
5.0,273,276
5.0,273,278
4.0,280,281
5.0,280,283
5.0,280,285
5.0,280,287
5.0,280,289
5.0,280,291
5.0,280,293


## Item Based Collaborative Filtering - Recommender System
##### In questo sottoparagrafo proviamo ad utilizzare l'item based recommender system sul dataset, inoltre proviamo a buildare il modello utilizzando dei parametri differenti per testare l'efficacia del sistema

In [9]:
item_based_cf_model = tc.recommender.item_similarity_recommender.create(sf, user_id = 'user_id' , item_id = 'item_id' , 
                                                          target = 'user_rating' , user_data = None ,
                                                          item_data = None , nearest_items = None , similarity_type = 'jaccard' ,
                                                          threshold = 0.001, verbose = False)
    
ibcf_similarity = item_based_cf_model.get_similar_items(k=3, verbose = False)
ibcf_similarity.print_rows(num_rows=20, num_columns=4)
ibcf_recommend = item_based_cf_model.recommend(k=3, verbose = False)
ibcf_recommend.print_rows(num_rows=20, num_columns=4)


+---------+---------+---------------------+------+
| item_id | similar |        score        | rank |
+---------+---------+---------------------+------+
|   280   |   424   | 0.09302324056625366 |  1   |
|   280   |   558   |  0.0810810923576355 |  2   |
|   280   |   536   | 0.07894736528396606 |  3   |
|   736   |   1294  | 0.19999998807907104 |  1   |
|   736   |   424   |  0.0476190447807312 |  2   |
|   736   |   280   |       0.03125       |  3   |
|   424   |   558   |  0.2083333134651184 |  1   |
|   424   |   611   | 0.12000000476837158 |  2   |
|   424   |   536   |  0.1111111044883728 |  3   |
|   485   |   736   |  0.1111111044883728 |  1   |
|   485   |   424   | 0.09090906381607056 |  2   |
|   485   |   611   | 0.06666666269302368 |  3   |
|   611   |   558   | 0.23529410362243652 |  1   |
|   611   |   1686  |  0.1818181872367859 |  2   |
|   611   |   424   | 0.12000000476837158 |  3   |
|   1329  |   1302  |         1.0         |  1   |
|   1329  |   536   | 0.0833333

## User Based Collaborative Filtering - Recommender System
##### In questo sottoparagrafo proviamo ad utilizzare l'user based recommender system sul dataset, utilizzeremo la stessa tipologia di modello utilizzata in precedenza ma con i parametri user_id e item_id invertiti come suggerisce la documentazione. Inoltre, come fatto nel caso precedente,  testiamo anche diversi parametri per valutare il sistema con l'efficacia migliore

In [5]:
user_based_cf_model = tc.recommender.item_similarity_recommender.create(sf, user_id = 'item_id' , item_id = 'user_id' , 
                                                          target = 'user_rating' , user_data = None ,
                                                          item_data = None , nearest_items = None , similarity_type = 'cosine' ,
                                                          threshold = 0.001, verbose = False)
    
ubcf_similarity = user_based_cf_model.get_similar_items(k=5, verbose = False)
ubcf_similarity.print_rows(num_rows=20, num_columns=4)
ubcf_recommendation = user_based_cf_model.recommend(k=5, verbose = False)
ubcf_recommendation.print_rows(num_rows=20, num_columns=4)


+---------+---------+-------+------+
| user_id | similar | score | rank |
+---------+---------+-------+------+
|   274   |   276   |  1.0  |  1   |
|   274   |   278   |  1.0  |  2   |
|   276   |   274   |  1.0  |  1   |
|   276   |   278   |  1.0  |  2   |
|   278   |   274   |  1.0  |  1   |
|   278   |   276   |  1.0  |  2   |
|   281   |   283   |  1.0  |  1   |
|   281   |   285   |  1.0  |  2   |
|   281   |   287   |  1.0  |  3   |
|   281   |   289   |  1.0  |  4   |
|   281   |   291   |  1.0  |  5   |
|   283   |   281   |  1.0  |  1   |
|   283   |   285   |  1.0  |  2   |
|   283   |   287   |  1.0  |  3   |
|   283   |   289   |  1.0  |  4   |
|   283   |   291   |  1.0  |  5   |
|   285   |   281   |  1.0  |  1   |
|   285   |   283   |  1.0  |  2   |
|   285   |   287   |  1.0  |  3   |
|   285   |   289   |  1.0  |  4   |
+---------+---------+-------+------+
[1676 rows x 4 columns]

+---------+---------+---------------------+------+
| item_id | user_id |        score  

## Matrix Factorization Based Recommender System
##### In questo sottoparagrafo proviamo ad utilizzare la matrix factorization sul dataset, alternativa il cui approccio alla base risulta simile al Collaborative Filtering. Proviamo anche qui a far runnare il modello con diversi parametri

In [19]:
matrix_factorization_model = tc.ranking_factorization_recommender.create(sf, target = 'user_rating' ,
                                                         ranking_regularization = 0.1 ,
                                                         unobserved_rating_value = 1,
                                                         max_iterations = 50, solver = "sgd", 
                                                         verbose = False)
s = matrix_factorization_model.recommend(verbose = False)
s.print_rows(num_rows=20, num_columns=4)
t = matrix_factorization_model.get_similar_items(k = 5, verbose = False)
t.print_rows(num_rows=20, num_columns=4)

+---------+---------+--------------------+------+
| user_id | item_id |       score        | rank |
+---------+---------+--------------------+------+
|   274   |   379   | 5.272665742548293  |  1   |
|   274   |   424   | 5.075950715454406  |  2   |
|   274   |   665   | 4.890501510115451  |  3   |
|   274   |   744   |  4.84343478432495  |  4   |
|   274   |   280   | 4.828578199023491  |  5   |
|   274   |   558   | 4.778087783368415  |  6   |
|   274   |   536   | 4.765501100333518  |  7   |
|   274   |   961   |  4.69032889953453  |  8   |
|   274   |   496   | 4.665346074612922  |  9   |
|   274   |   884   | 4.664351153882331  |  10  |
|   276   |   379   | 5.298867318542785  |  1   |
|   276   |   424   | 5.0988431905825795 |  2   |
|   276   |   665   | 4.927590373667067  |  3   |
|   276   |   280   | 4.894263025136775  |  4   |
|   276   |   744   | 4.863327255236095  |  5   |
|   276   |   558   | 4.814578208835906  |  6   |
|   276   |   536   | 4.7812662383039655 |  7   |


## Content Based Recommender System
##### In questo sottoparagrafo proviamo ad utilizzare un Content Based Recommender sul dataset, in particolare lo faremo sul file trails.csv che contiene tutte le feature piu' importanti di un percorso (circa 60 in totale)

In [16]:
trails = tc.SFrame.read_csv('trails.csv', verbose = False)
trails = trails.rename({'trail_id': 'item_id'}) #rinomiano il trail_id in item_id in maniera tale da conformarci con la libreria 
content_based_model = tc.recommender.item_content_recommender.create(item_data = trails, item_id="item_id", 
                                                                     observation_data = sf, user_id="user_id", 
                                                                     target="user_rating", verbose = False)
content_based_similiarity = content_based_model.get_similar_items(verbose = False)
content_based_similiarity.print_rows(num_rows=20, num_columns=4)
content_based_recommend = content_based_model.recommend(verbose = False)
content_based_recommend.print_rows(num_rows=20, num_columns=4)

+---------+---------+--------------------+------+
| item_id | similar |       score        | rank |
+---------+---------+--------------------+------+
|   273   |   736   | 0.9199782013893127 |  1   |
|   273   |   1194  | 0.9199226498603821 |  2   |
|   273   |   1720  | 0.919810950756073  |  3   |
|   273   |   1533  | 0.8998916149139404 |  4   |
|   273   |   558   | 0.8997915387153625 |  5   |
|   273   |   485   | 0.8994442820549011 |  6   |
|   273   |   1313  | 0.8993611335754395 |  7   |
|   273   |   1686  | 0.8978621363639832 |  8   |
|   273   |   536   | 0.8973095417022705 |  9   |
|   273   |   1623  | 0.8813763856887817 |  10  |
|   280   |   925   | 0.9575408697128296 |  1   |
|   280   |   611   | 0.954775333404541  |  2   |
|   280   |   558   | 0.9193320274353027 |  3   |
|   280   |   536   | 0.9178102016448975 |  4   |
|   280   |   1477  | 0.8980252742767334 |  5   |
|   280   |   485   | 0.8784804940223694 |  6   |
|   280   |   1686  | 0.8599282503128052 |  7   |


## Popular Based Recommender System
##### In questo sottoparagrafo proviamo ad utilizzare il Recommender system piu' semplice: il popular based. Le raccomandazioni fornite sono utilii per superare il problema del Cold Start e per spingere l'utente a recensire piu' item possibili al fine di ottenere in futuro delle raccomandazioni che siano il piu' possibile pertinenti.

In [18]:
popular_based_model = tc.popularity_recommender.create(sf, target = 'user_rating', verbose = False)
popresult = popular_based_model.recommend()
popresult.print_rows(num_rows=20, num_columns=4)

+---------+---------+-------+------+
| user_id | item_id | score | rank |
+---------+---------+-------+------+
|   274   |   1710  |  5.0  |  1   |
|   274   |   1259  |  5.0  |  2   |
|   274   |   914   |  5.0  |  3   |
|   274   |   1533  |  5.0  |  4   |
|   274   |   1684  |  5.0  |  5   |
|   274   |   417   |  5.0  |  6   |
|   274   |   376   |  5.0  |  7   |
|   274   |   347   |  5.0  |  8   |
|   274   |   1190  |  5.0  |  9   |
|   274   |   830   |  5.0  |  10  |
|   276   |   1710  |  5.0  |  1   |
|   276   |   1259  |  5.0  |  2   |
|   276   |   914   |  5.0  |  3   |
|   276   |   1533  |  5.0  |  4   |
|   276   |   1684  |  5.0  |  5   |
|   276   |   417   |  5.0  |  6   |
|   276   |   376   |  5.0  |  7   |
|   276   |   347   |  5.0  |  8   |
|   276   |   1190  |  5.0  |  9   |
|   276   |   830   |  5.0  |  10  |
+---------+---------+-------+------+
[4700 rows x 4 columns]

